# 1. Import the required libraries

In [ ]:
# Standard code libraries
import os
import ipywidgets as widgets

# Custom code libraries from ReSurfEMG
from resurfemg.data_connector.config import Config
from resurfemg.data_connector.file_discovery import find_files, find_folders
from resurfemg.pipelines.ipy_widgets import file_select
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader

config = Config()

%matplotlib widget

## 2. Load sEMG test data

In [2]:
# For the test data:
base_path = config.get_directory('test_data')

files = find_files(
    base_path=base_path,
    file_name_regex='*',
    extension_regex='Poly5',
    folder_levels=None,
    verbose=False
)
button_list = file_select(
    files=files,
    folder_levels=['files'],
    default_value_select=None,
    default_idx_select=[0])

interactive(children=(Dropdown(description='files:', options=('emg_data_synth_pocc.Poly5', 'emg_data_synth_qui…

In [3]:
# Load the EMG from the selected folders:
emg_file_chosen = os.path.join(base_path, *[btn.value for btn in button_list])

data_emg = Poly5Reader(emg_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate

Reading file  C:/Users/warnaarrsp/repos/ReSurfEMG_ReSurfEMG/test_data\emg_data_synth_pocc.Poly5
	 Number of samples:  860160 
	 Number of channels:  2 
	 Sample rate: 2048 Hz
Done reading data.


## 3. Exammple: TOPSPIN data sEMG test data

Example for the TOPSPIN study data, which is structured according to:
- base_path/
   - patients/
       - date/
           - measurement/
               - EMG_recording.Poly5
               - Draeger_recording.Poly5

In [4]:
# First find the patients
base_path = config.get_directory('patient_data')
patient_paths = find_folders(
    base_path,
    folder_levels=['patient'])
patient_paths = list(patient_paths['patient'].values)

patient = widgets.Dropdown(
    options=patient_paths,
    description='Patient' + ':',
    disabled=False,
)
display(patient)

ValueError: Specified base_path cannot be found.

In [ ]:
# Then find the files for the selected patients:
#   NB One could also run this with:
#       base_path=base_path, 
#       folder_levels = ['patient', date', 'measurement']
#   This just takes way longer as the find_files has to iterate over all
#   folders and files
file_name_regex = None
extension_regex = 'poly5'
folder_levels = ['date', 'measurement']
patient_path = os.path.join(base_path, patient.value)

files = find_files(
    base_path=patient_path,
    file_name_regex=file_name_regex,
    extension_regex=extension_regex,
    folder_levels=folder_levels)

# Then generate the linked buttons for the find files up to the measurement
# level:
button_list = file_select(
    files,
    folder_levels=folder_levels[:-1],
    default_value_select=[None, '009'],
    default_idx_select=[0, None])

In [ ]:
# Load the EMG from the selected folders:
emg_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'EMG_recording.Poly5')

data_emg = Poly5Reader(emg_file_chosen)
y_emg = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate

# Load the ventilator from the selected folders:
vent_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'Draeger_recording.Poly5')

data_vent = Poly5Reader(vent_file_chosen)
y_vent = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate
